# Example: CitiBike data

Adapted from Kelsey Jordahl
https://gist.github.com/kjordahl/5957573

In [1]:
import requests  
import json
from pyproj import Proj
from shapely.geometry import Point
import geopandas as gp

NYC borough boundaries downloaded from [Bytes of the Big Apple](http://www.nyc.gov/html/dcp/download/bytes/nybb_13a.zip)

In [2]:
boros = gp.GeoDataFrame.from_file('nybb_15b/nybb.shp')

Load real time bike station data from [CitiBike](http://citibikenyc.com) json API:

In [3]:
# get the bike data and convert it to a ditcionary
endpoint_url = 'http://citibikenyc.com/stations/json'
response = requests.get(endpoint_url)
data = json.loads(response.text)


In [4]:
# convert the relevant part to a geodataframe
df = gp.GeoDataFrame(data['stationBeanList'])
df.head()

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,29,25,,72,,2019-03-07 11:51:58 PM,40.767272,,-73.993929,,W 52 St & 11 Ave,,W 52 St & 11 Ave,1,In Service,False,55
1,,18,14,,79,,2019-03-07 11:51:26 PM,40.719116,,-74.006667,,Franklin St & W Broadway,,Franklin St & W Broadway,1,In Service,False,33
2,,20,7,,82,,2019-03-07 11:54:34 PM,40.711174,,-74.000165,,St James Pl & Pearl St,,St James Pl & Pearl St,1,In Service,False,27
3,,35,27,,83,,2019-03-07 11:54:46 PM,40.683826,,-73.976323,,Atlantic Ave & Fort Greene Pl,,Atlantic Ave & Fort Greene Pl,1,In Service,False,62
4,,12,7,,119,,2019-03-07 11:51:21 PM,40.696089,,-73.978034,,Park Ave & St Edwards St,,Park Ave & St Edwards St,1,In Service,False,19


In [5]:
# there is one row for each bike station.  How many stations are there? 
len(df)

817

In [9]:
# in the file above, there are lon-lats, but no geometry field
# we need to set that up

s = gp.GeoSeries([Point(x, y) for x, y in zip(df['longitude'], df['latitude'])])
df['geometry'] = s
df.crs = {'init': 'epsg:4326', 'no_defs': True}
df.geometry.total_bounds

array([-74.0836394 ,  40.65539977, -73.9077436 ,  40.81439444])

In [10]:
# make sure they are on the same CRS.  
# checking the bounds is a nice way of seeing this
df.to_crs(boros.crs, inplace=True)
df.geometry.total_bounds

array([ 961064.73659426,  178056.37414829, 1009803.53178432,
        235986.42166716])

In [11]:
# the geometry objects can do lots of cool stuff.  For example: 

manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'stations in Manhattan')

418 stations in Manhattan


# Your turn

You can read about the range of operations available in geopandas here: 
    
http://geopandas.org/index.html

Your assignment is to: 

1. Calculate how many stations are in each borough
2. Calculate how many bikes are currently available in each borough
3. Read about and try at least two new spatial or geometric operations (beyond what I've covered here). 

In [16]:
# 1. Stations in each borough

# Manhattan
manhattan = boros.geometry[3]
m_sta = df.geometry.within(manhattan)

# Bronx
bronx = boros.geometry[4]
bx_sta = df.geometry.within(bronx)

# Brooklyn
brooklyn = boros.geometry[1]
by_sta = df.geometry.within(brooklyn)

# Queens
queens = boros.geometry[2]
q_sta = df.geometry.within(queens)

# Stanten Island
stanten_island = boros.geometry[0]
si_sta = df.geometry.within(stanten_island)

print (sum(m_sta), 'stations in Manhattan')
print (sum(bx_sta), 'stations in Bronx')
print (sum(by_sta), 'stations in Brooklyn')
print (sum(q_sta), 'stations in Queens')
print (sum(si_sta), 'stations in Staten Island')

418 stations in Manhattan
0 stations in Bronx
267 stations in Brooklyn
82 stations in Queens
0 stations in Staten Island


In [17]:
# 2. Calculate how many bikes are available in each borough

print (sum(df['availableBikes'][m_sta]), 'bikes available in Manahattan')
print (sum(df['availableBikes'][bx_sta]), 'bikes available in the Bronx')
print (sum(df['availableBikes'][by_sta]), 'bikes available in Brooklyn')
print (sum(df['availableBikes'][q_sta]), 'bikes available in Queens')
print (sum(df['availableBikes'][si_sta]), 'bikes available in Staten Island')

6745 bikes available in Manahattan
0 bikes available in the Bronx
4102 bikes available in Brooklyn
427 bikes available in Queens
0 bikes available in Staten Island
